# 从网页中截取文章内容

## 步骤1：参数设置

In [33]:
url = 'http://blog.csdn.net/roseki/article/details/72571459'
print(url)

http://blog.csdn.net/roseki/article/details/72571459


## 步骤2：查找相应的截取关键字

In [34]:
if 'blog.jobbole.com' in url:  # 伯乐在线
    title_key = '.entry-header'
    content_key = '.entry'
    print('伯乐在线 http://blog.jobbole.com')
elif 'blog.csdn.net' in url:  # csdn
    title_key = '.link_title'
    content_key = '#article_content'
    print('csdn http://blog.csdn.net')
elif 'www.codingpy.com' in url:  # 编程派网址
    title_key = '.header h1'
    content_key = '.article-content'
    print('编程派网址 http://www.codingpy.com')
else:
    title_key = '#keraspython'
    content_key = '.section'
    print('其它')

csdn http://blog.csdn.net


## 步骤3：发出网页请求，接收响应

In [35]:
from urllib import request
req = request.Request(url)
res = request.urlopen(req)
html = res.read().decode('utf-8')
print(html)


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">
    
    <html xmlns="http://www.w3.org/1999/xhtml">
    
<head>          
            <link rel="canonical" href="http://blog.csdn.net/roseki/article/details/72571459"/> 

   

  
 <meta http-equiv="Cache-Control" content="no-siteapp" /><link rel="alternate" media="handheld" href="#" />

    <meta name="shenma-site-verification" content="5a59773ab8077d4a62bf469ab966a63b_1497598848"> 
       
    <title>罗斯基白话：TensorFlow + 实战系列（五）实战MNIST - roseki的博客
        - CSDN博客</title>
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
    <meta name="description" content="实战MNIST手写数字识别" />

    <meta name="keywords" content="TensorFlow实战,深度学习,优化" />

   
    <script src="http://static.blog.csdn.net/scripts/blog_static_head.min.js" type="text/javascript"></script>

    
        <!--new top-->
       
       
        <!--new top-->
    
      <!-- ad begin -->
 

## 步骤4：从网页中提取文章标题和内容

In [36]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')
title = soup.select(title_key)[0].text.strip()  # 文章标题
print('文章标题：', title)
content = soup.select(content_key)[0]  # 文章内容

文章标题： 罗斯基白话：TensorFlow + 实战系列（五）实战MNIST


## 步骤5：下载文章中的图片

### 5.1 哈希码生成函数，用于给图片重新命名

In [37]:
import hashlib
def md5(name):
    """ 将字符串转成哈希码 """
    if not isinstance(name, str):
        name = str(name)
    md5 = hashlib.md5()
    md5.update(name.encode('utf-8'))
    return md5.hexdigest()

### 5.2 下载图片，并修改文章图片的超链接

In [38]:
import re
import os
content = str(content)
pattern = '<img .*?src=\"(.*?)\"'
re_image = re.compile(pattern)
for image_link in re_image.findall(content):
    if not os.path.exists('images'):
        os.mkdir('images')
    filename = 'images/' + md5(image_link) + os.path.splitext(image_link)[-1]
    try:
        request.urlretrieve(image_link, filename)
        print('下载完成', filename)
    except Exception as e:
        print('图片出错', e)
    else:
        content = content.replace(image_link, filename)
print('已完成--------')

下载完成 images/6bcd23e95843b8b0735c0bdd29210517
下载完成 images/4fb6655622cace3e633ef51840d5c5b2
下载完成 images/5fdd0feb1b37011649bf28930a76e3dd
下载完成 images/10d27ff6b14262c0cf1fa8ff39e6abbc
下载完成 images/a7a3f0210e7b8a1e4e709a79473eecc9
已完成--------


## 步骤6：将截取的文章标题和内容重新组合成新的网页文件

In [39]:
html_template = """<!DOCTYPE html>
<html><head><meta charset="UTF-8">
</head><body>
<p><a href="{origin}">原文链接</a></p>
<p><center><h1>{title}</h1></center></p>
    {content}
</body></html>"""
html = html_template.format(origin=url, title=title, content=content)
print(html)

<!DOCTYPE html>
<html><head><meta charset="UTF-8">
</head><body>
<p><a href="http://blog.csdn.net/roseki/article/details/72571459">原文链接</a></p>
<p><center><h1>罗斯基白话：TensorFlow + 实战系列（五）实战MNIST</h1></center></p>
    <div class="article_content tracking-ad" data-dsm="post" data-mod="popu_307" id="article_content">
<p> </p>
<h1 style="text-align:center"><span style="color:#ff6666">白话TensorFlow +实战系列（五）<br>
实战MNIST</br></span></h1>
<p> </p>
<p><span style="font-size:18px">       这篇文章主要用全连接神经网络来实现MNIST手写数字识别的问题。首先介绍下MNIST数据集。</span></p>
<p><span style="font-size:18px">       1）MNIST数据集</span></p>
<p><span style="font-size:18px">       MNIST数据集是一个非常有名的手写数字识别数据集，它包含了60000张图片作为训练集，10000张图片为测试集，每张图为一个手写的0~9数字。如图：</span></p>
<p><br>
</br></p>
<p style="text-align:center"><img alt="" src="images/6bcd23e95843b8b0735c0bdd29210517"><br>
</br></img></p>
<p></p>
<p><br>
</br></p>
<p><span style="font-size:18px">其中每张图的大小均为28*28，这里大小指的的是像素。例如数字1所对应的像素矩阵为：</span></p>
<p><br>
</br></p>
<p style="text-alig

## 步骤7：将文件写入磁盘

In [40]:
filename = title + ".html"
with open(filename, "w") as f:
    f.write(html.replace(u'\xa0', u' ').replace(u'\U0001f60a', u' '))  # 在windows中出错，所以这里进行了字符串替换

## (完)